In [29]:
import sys
import json
import pandas as pd
import numpy as np
import os
import boto3
from decimal import Decimal
from math import pi

ExpressionAttributeNames = {
    "#TRP": "trip",
    "#TM": "tm",
    "#RPM": "rpm",
    "#SPD": "speed",
    "#THRTL": "throttle_pos",
    "#ACC_X": "acc_x",
    "#ACC_Y": "acc_y",
    "#ACC_Z": "acc_z",
    "#GYRO_X": "gyro_x",
    "#GYRO_Y": "gyro_y",
    "#GYRO_Z": "gyro_z",
    "#LONG": "long",
    "#LAT": "lat",
    "#DELTA_ACC_X": "delta_acc_x",
    "#DELTA_ACC_Y": "delta_acc_y",
    "#DELTA_ACC_Z": "delta_acc_z",
}

ProjectionExpression = "#TRP, #TM, #RPM, #SPD, #THRTL, #ACC_X, #ACC_Y, #ACC_Z, #GYRO_X, #GYRO_Y, #GYRO_Z, #DELTA_ACC_X, #DELTA_ACC_Y, #DELTA_ACC_Z, #LONG, #LAT"

# Returns a processed dataframe that contains the information needed for the heatmap
# :return: a pandas DataFrame that contains the information needed for the heatmap
def get_data(event):
    dynamodb = boto3.resource('dynamodb')
    table = dynamodb.Table('vehicle_data')
    trip_id = event["trip_id"]
        
    response = table.scan(ExpressionAttributeNames = ExpressionAttributeNames, ProjectionExpression = ProjectionExpression)
    data = pd.DataFrame.from_dict(response['Items'])

    return data[data["trip"] == trip_id]



In [30]:
def driver_profile(file):
    trip_time, col_count = file.shape
    diff_list = []
    wreck_points = 0
    sharp_steer = 0
    for x, y in zip(file['speed'][0::], file['speed'][1::]):
        if (abs(y-x) > 2):
            wreck_points += 1
        diff_list.append(y-x)
    
    # printing difference list
    count_length = trip_time - 1
    percent_wreck = (wreck_points/count_length)*100
    #print ("difference list: ", str(diff_list))
    print("Amount of total trip that identified major difference in speeds per timestamp:",percent_wreck)
    
    
    ## average speed per cluster of 5
    sum_list = []
    for i in range(0,count_length, 5):
       sum_list.append(sum(file['speed'][i:i+5])/5)
    # print(sum_list[0:10])
    
    ## maximum speed observed 
    print("Maximum Speed Observed:",max(file['speed']))
    
    ## identify sudden brakes 
    #according to research, deceleration of < 15 fps is considered safe
    jerk_break_count = 0
    for x, y in zip(file['speed'][0::], file['speed'][1::]):
        x = x * Decimal(1.467)
        y = y * Decimal(1.467)
        if ((y-x) < 0 and abs(y-x) > 7): #use 7 bc there was 0 occurence of >15
            jerk_break_count +=1
    print("Percent of sudden brake instances indentified:",(jerk_break_count/count_length)*100)
    
    ## is the driver displaying stable speeding?
    unstable_count = 0
    for x, y in zip(file['speed'][0::], file['speed'][1::]):
        if (abs(y-x)>1.5):
            unstable_count +=1
    print("Percent of trip with inconsistent speeding: ",round((unstable_count/count_length)*100,2))
    
    ## Number of stops made during trips
    stop_count = 0
    for x, y in zip(file['speed'][0::], file['speed'][1::]):
        if (y == 0 and abs(y-x) > 0.6):
            stop_count +=1
    print("Number of stops made during this trip: ", round(stop_count,2))

    return [
        "Amount of total trip that identified major difference in speeds per timestamp: {}".format(percent_wreck),
        "Maximum Speed Observed: {}".format(max(file['speed'])),
        "Percent of sudden brake instances indentified: {}".format((jerk_break_count/count_length)*100),
        "Percent of trip with inconsistent speeding: {}".format(round((unstable_count/count_length)*100,2)),
        "Number of stops made during this trip: {}".format(round(stop_count,2))
    ]

In [34]:
get_data({'trip_id': 6})

,acc_y,acc_z,trip,acc_x,gyro_z,rpm,gyro_y,throttle_pos,gyro_x,delta_acc_z,delta_acc_x,delta_acc_y,tm,speed,lat,long
2241,-0.6943184814453125,10.127473022460936,6,-0.9241618408203124,0.0644839222290076,898.25,0.0079938746564885,12.15686274509804,0.0572894350381679,0.3974374755859369,-1.388636962890625,-2.9735984619140625,1649379164,26.097590073968032,30.601981666666667,-96.339075
2242,0.689530078125,9.595960253906249,6,0.05746083984375,0.0311761111603053,905.25,-0.1660061303664122,12.549019607843135,0.2222297154503817,-0.5315127685546877,0.9816226806640624,1.3838485595703125,1649379165,24.85484768949336,30.60208833333333,-96.33912333333336
2243,-1.70467158203125,11.458649145507811,6,-2.681505859375,0.0095926495877862,906.25,0.0426339981679389,12.15686274509804,0.113779482610687,1.8626888916015627,-2.73896669921875,-2.39420166015625,1649379166,24.233476497256028,30.602191666666663,-96.33916666666669
2244,-0.6607996582031249,9.323021264648435,6,1.0295067138671874,-0.0170535992671755,868.75,-0.0255803989007633,12.549019607843135,-0.0602205224122137,-2.135627880859376,3.7110125732421873,1.0438719238281249,1649379167,23.61210530501869,30.602291666666662,-96.33920833333336
2245,0.229843359375,10.0748005859375,6,0.469263525390625,0.0221163865496183,883.5,-0.0141225118931297,12.15686274509804,0.0538254226870229,0.7517793212890638,-0.5602431884765624,0.8906430175781249,1649379168,22.369362920544027,30.602386666666668,-96.33925166666668
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3890,-0.181959326171875,10.012551342773437,6,1.1827356201171877,0.0157212868244274,645,0.0045298623053435,11.764705882352942,0.0093261870992366,-0.14844050292968802,0.009576806640625213,-0.11013327636718749,1649380816,0,30.610795,-96.35527833333332
3891,-0.1963245361328124,10.127473022460936,6,1.197100830078125,0.0159877493129771,645,-0.0010658499541984,11.764705882352942,0.0103920370534351,0.11492167968749989,0.014365209960937264,-0.014365209960937403,1649380817,0,30.61079333333333,-96.35527833333332
3892,-0.1484405029296875,10.09395419921875,6,1.1444283935546875,0.0093261870992366,645,0.00692802470229,11.764705882352942,0.0157212868244274,-0.03351882324218636,-0.05267243652343745,0.0478840332031249,1649380818,0,30.61079333333333,-96.35527833333332
3893,-0.201112939453125,10.137049829101562,6,1.1444283935546875,0.0119908119847328,645,0.0007993874656488,11.764705882352942,0.0109249620305343,0.04309562988281179,0,-0.052672436523437505,1649380819,0,30.610795,-96.35527833333332


In [35]:
def lambda_handler(event, context):
    data = get_data(event)
    avg_v = data.speed.mean()
    avg_a = np.sqrt(data.acc_x.mean() ** 2 + data.acc_y.mean() ** 2 + data.acc_z.mean() ** 2)
    trip_tm = data.tm.iloc[-1] - data.tm.iloc[0]
    avg_rpm = data.rpm.mean()
    # time = json.dumps(list(pd.to_datetime(data['tm'].astype(float),unit='s')))
    time = json.dumps(list((pd.to_datetime(data['tm'].astype(float),unit='s')).astype(str)))
    speed = json.dumps(list(round(data.speed.astype(float), 2)))
    throttle_pos = json.dumps(list(round(data.throttle_pos.astype(float), 3)))
    longi = json.dumps(list(data['long'].astype(float)))
    lat = json.dumps(list(data.lat.astype(float)))
    delta_acc_x = json.dumps(list(data.delta_acc_x.astype(float)))
    delta_acc_y = json.dumps(list(data.delta_acc_y.astype(float)))
    delta_acc_z = json.dumps(list(data.delta_acc_z.astype(float)))

    analysis_results = driver_profile(data)

    return {
        'statusCode': 200,
        'headers': {'Content-Type': 'application/json', 'Access-Control-Allow-Origin': '*', 'Access-Control-Allow-Headers': 'Access-Control-Allow-Origin'},
        'body': {"average_speed": json.dumps(avg_v), "average_acceleration": json.dumps(avg_a), 
        "average_rpm": json.dumps(float(avg_rpm)), "trip_time": json.dumps(float(trip_tm)), "time": time, "speed":speed, 
        "throttle_position":throttle_pos, "long":longi, "lat":lat, "delta_acc_x":delta_acc_x, "delta_acc_y":delta_acc_y, "delta_acc_z":delta_acc_z,
        "video_link": json.dumps('https://vboxbucket.s3.amazonaws.com/{}.mp4'.format(event["trip_id"])),
        "analysis_results": json.dumps(analysis_results)
        }
    }


In [36]:
lambda_handler(event = {'trip_id':5}, context = 0)

Amount of total trip that identified major difference in speeds per timestamp: 17.08185053380783
Maximum Speed Observed: 57.16614968583473
Percent of sudden brake instances indentified: 0.0
Percent of trip with inconsistent speeding:  24.2
Number of stops made during this trip:  5


{'statusCode': 200,
 'headers': {'Content-Type': 'application/json',
  'Access-Control-Allow-Origin': '*',
  'Access-Control-Allow-Headers': 'Access-Control-Allow-Origin'},
 'body': {'average_speed': '23.748718829553145',
  'average_acceleration': '10.187128792099601',
  'average_rpm': '1198.6329787234042',
  'trip_time': '282.0',
  'time': '["2022-04-18 00:57:59", "2022-04-18 00:58:00", "2022-04-18 00:58:01", "2022-04-18 00:58:02", "2022-04-18 00:58:03", "2022-04-18 00:58:04", "2022-04-18 00:58:05", "2022-04-18 00:58:06", "2022-04-18 00:58:07", "2022-04-18 00:58:08", "2022-04-18 00:58:09", "2022-04-18 00:58:10", "2022-04-18 00:58:11", "2022-04-18 00:58:12", "2022-04-18 00:58:13", "2022-04-18 00:58:14", "2022-04-18 00:58:15", "2022-04-18 00:58:16", "2022-04-18 00:58:17", "2022-04-18 00:58:18", "2022-04-18 00:58:19", "2022-04-18 00:58:20", "2022-04-18 00:58:21", "2022-04-18 00:58:22", "2022-04-18 00:58:23", "2022-04-18 00:58:24", "2022-04-18 00:58:25", "2022-04-18 00:58:26", "2022-04-18

In [6]:
import requests
# myHeaders = {"Content-Type", "application/json"}
# response = requests.get("https://7m3q3dqm4g.execute-api.us-east-1.amazonaws.com/dev", headers=myHeaders)
# print(response)

headers = {}
url = "https://7m3q3dqm4g.execute-api.us-east-1.amazonaws.com/dev"
method = 'POST'
redirect = 'follow'
body={"trip_id": 6}


response = requests.request(method=method, url=url, data = json.dumps(body), headers=headers)
print(json.loads(response.text))


{'statusCode': 200, 'headers': {'Content-Type': 'application/json', 'Access-Control-Allow-Origin': '*', 'Access-Control-Allow-Headers': 'Access-Control-Allow-Origin'}, 'body': {'average_speed': '25.91839173134578', 'average_acceleration': '10.177949840888145', 'average_rpm': '1218.84915356711', 'trip_time': '1656.0', 'time': '["2022-04-08 00:52:44", "2022-04-08 00:52:45", "2022-04-08 00:52:46", "2022-04-08 00:52:47", "2022-04-08 00:52:48", "2022-04-08 00:52:49", "2022-04-08 00:52:50", "2022-04-08 00:52:51", "2022-04-08 00:52:52", "2022-04-08 00:52:53", "2022-04-08 00:52:54", "2022-04-08 00:52:55", "2022-04-08 00:52:56", "2022-04-08 00:52:57", "2022-04-08 00:52:58", "2022-04-08 00:52:59", "2022-04-08 00:53:00", "2022-04-08 00:53:01", "2022-04-08 00:53:02", "2022-04-08 00:53:03", "2022-04-08 00:53:04", "2022-04-08 00:53:05", "2022-04-08 00:53:06", "2022-04-08 00:53:07", "2022-04-08 00:53:08", "2022-04-08 00:53:09", "2022-04-08 00:53:10", "2022-04-08 00:53:11", "2022-04-08 00:53:12", "202

In [7]:
dc = json.loads(response.text)
dc["body"]["lat"]

'[30.601981666666667, 30.60208833333333, 30.602191666666663, 30.602291666666662, 30.602386666666668, 30.60248, 30.602568333333338, 30.602656666666668, 30.602744999999995, 30.602834999999995, 30.602921666666663, 30.60300833333333, 30.60309833333333, 30.60318833333333, 30.603283333333334, 30.60337666666667, 30.603465, 30.603546666666663, 30.60361833333333, 30.60367833333333, 30.603728333333333, 30.60376666666667, 30.603790000000004, 30.6038, 30.603801666666666, 30.6038, 30.60379833333333, 30.603795, 30.603793333333336, 30.603791666666663, 30.603791666666663, 30.603791666666663, 30.603791666666663, 30.603793333333336, 30.603795, 30.603795, 30.603793333333336, 30.603793333333336, 30.603791666666663, 30.603791666666663, 30.603790000000004, 30.603790000000004, 30.60378833333333, 30.60378833333333, 30.60378833333333, 30.60378833333333, 30.60378833333333, 30.60378833333333, 30.603790000000004, 30.603790000000004, 30.603790000000004, 30.603790000000004, 30.603790000000004, 30.603790000000004, 3